**Лабораторная работа №4**

***Задание:***

1) Найти оптимум тестовых функций для оптимизации с помощью Optuna

2) Осуществить подбор параметров модели классификации с помощью Optuna

Импортируем библиотеки

In [1]:
!pip install optuna
import optuna
import numpy as np
import logging
import sys
import sklearn.datasets
import sklearn.linear_model
import sklearn.model_selection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 4.2 MB/s 
     |████████████████████████████████| 210 kB 54.9 MB/s 
     |████████████████████████████████| 81 kB 8.9 MB/s 
     |████████████████████████████████| 78 kB 6.9 MB/s 
     |████████████████████████████████| 50 kB 5.6 MB/s 
     |████████████████████████████████| 147 kB 52.6 MB/s 
     |████████████████████████████████| 112 kB 49.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=046ee830d3a9e7eec72c7c18f6ffb971f5772f53310cd0c89699f18901862c8a
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Successfully uninstalled importlib-metadata-5.1.0


**Функция Химмельблау**

In [3]:
def Himmelblau(x: np.float32, y: np.float32) -> np.float64:
    '''
    Функция Химмельблау
    
    Args:
        x(np.ndarray): Аргумент 1
        y(np.ndarray): Аргумент 2
        
    Returns:
        np.float64: Результат функции
    '''
    return np.square(np.square(x)+y-11)+np.square(x+np.square(y)-7);

def objective(trial):
    x = trial.suggest_float("x", -10, 10)
    y = trial.suggest_float("y", -10, 10)
    return Himmelblau(x, y)

In [4]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2022-12-30 10:22:43,597] A new study created in memory with name: no-name-fbc7e1ed-f170-43a6-adeb-0e6c6ca17c9b
[I 2022-12-30 10:22:43,604] Trial 0 finished with value: 213.6552789442453 and parameters: {'x': 4.39375014967964, 'y': 3.3754697090361496}. Best is trial 0 with value: 213.6552789442453.
[I 2022-12-30 10:22:43,612] Trial 1 finished with value: 2588.170458484471 and parameters: {'x': 7.424608794634281, 'y': -5.7415542336312715}. Best is trial 0 with value: 213.6552789442453.
[I 2022-12-30 10:22:43,619] Trial 2 finished with value: 1221.1382142766913 and parameters: {'x': -1.338074074274818, 'y': 6.571387602460259}. Best is trial 0 with value: 213.6552789442453.
[I 2022-12-30 10:22:43,622] Trial 3 finished with value: 93.05701477742565 and parameters: {'x': 2.282721307986124, 'y': 3.761430132855308}. Best is trial 3 with value: 93.05701477742565.
[I 2022-12-30 10:22:43,626] Trial 4 finished with value: 112.1400226505233 and parameters: {'x': -4.170410390407351, 'y': -1.36101

In [5]:
best_params = study.best_params
found_x = best_params["x"]
found_y = best_params["y"]
print('f(3, 2) = 0')
print('f(-2.805118, 3.131312) = 0')
print('f(-3.779310, -3.283186) = 0')
print('f(3.584428, -1.848126) = 0')
print(f"Found x: {found_x}, Found y: {found_y}, Result: {Himmelblau(found_x, found_y)}")

f(3, 2) = 0
f(-2.805118, 3.131312) = 0
f(-3.779310, -3.283186) = 0
f(3.584428, -1.848126) = 0
Found x: 2.4496894311784305, Found y: 2.269754648016085, Result: 7.810671480096079


**Датасет sklearn (Digits)**

In [6]:
def objective(trial):
    digs = sklearn.datasets.load_digits()
    classes = list(set(digs.target))
    train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(
        digs.data, digs.target, test_size=0.25, random_state=0
    )

    alpha = trial.suggest_float("alpha", 1e-5, 1e-1, log=True)
    clf = sklearn.linear_model.SGDClassifier(alpha=alpha)

    for step in range(100):
        clf.partial_fit(train_x, train_y, classes=classes)

        # Report intermediate objective value.
        intermediate_value = 1.0 - clf.score(valid_x, valid_y)
        trial.report(intermediate_value, step)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.TrialPruned()

    return 1.0 - clf.score(valid_x, valid_y)

In [7]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study = optuna.create_study(pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=20)

[I 2022-12-30 10:22:52,548] A new study created in memory with name: no-name-4df82678-0827-494f-9088-c55728dcbe68


A new study created in memory with name: no-name-4df82678-0827-494f-9088-c55728dcbe68


[I 2022-12-30 10:22:53,819] Trial 0 finished with value: 0.03555555555555556 and parameters: {'alpha': 0.04701264437266902}. Best is trial 0 with value: 0.03555555555555556.


Trial 0 finished with value: 0.03555555555555556 and parameters: {'alpha': 0.04701264437266902}. Best is trial 0 with value: 0.03555555555555556.


[I 2022-12-30 10:22:55,083] Trial 1 finished with value: 0.06444444444444442 and parameters: {'alpha': 0.00045181374077340927}. Best is trial 0 with value: 0.03555555555555556.


...

Trial 17 pruned. 


[I 2022-12-30 10:23:03,345] Trial 18 pruned. 


Trial 18 pruned. 


[I 2022-12-30 10:23:03,483] Trial 19 pruned. 


Trial 19 pruned. 


In [8]:
#Графики
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [9]:
plot_optimization_history(study)

In [10]:
plot_intermediate_values(study)

In [11]:
plot_parallel_coordinate(study)

In [12]:
plot_slice(study)

In [13]:
plot_param_importances(study)

In [14]:
plot_edf(study)